#### feature engineer ==> libsvm

In [1]:
import pandas as pd
import numpy as np

one_hot_feature=['col_0', 'col_1', 'col_2', 'col_4']
train_file = "./data/train_nosampling.csv"
test_file = "./data/test_nosampling.csv"

train = pd.read_csv(train_file, names=['col_0', 'col_1', 'col_2', 'col_3', 'col_4'])
test = pd.read_csv(test_file, names=['col_0', 'col_1', 'col_2', 'col_3', 'col_4'])
train_ = train.head(3)
test_ = test.head(3)
#print(len(train_), len(test_))
data = pd.concat([train_, test_], axis=0)
data = data.reset_index(drop = True)
data = data[one_hot_feature]
#print(len(data))

In [2]:
class LibsvmFormat:
    def __init__(self,one_hot_feat):
        self.field_index_ = None
        self.feature_index_ = None
        self.one_hot_feat= one_hot_feat
        self.field_size = []

    def fit(self, df, y=None):
        self.field_index_ = {col: i for i, col in enumerate(df.columns)}
        self.feature_index_ = dict()
        last_idx = 0
        for col in df.columns:
            if col in self.one_hot_feat:
                print(col)
                vals = np.unique(df[col])
                self.field_size.append(len(vals))
                for val in vals:
                    if val==-1: continue
                    name = '{}_{}'.format(col, val)
                    # print (name)
                    if name not in self.feature_index_:
                        self.feature_index_[name] = last_idx
                        last_idx += 1
        print (self.field_size)
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df),self.field_size

    def transform_row_(self, row):
        ffm = []
        dic = sorted(row.loc[row != 0].to_dict().items(),key=lambda  item:int(item[0].split('_')[1]))
        # print (dic)
        for col, val in dic:
            if col in self.one_hot_feat:
                name = '{}_{}'.format(col, val)
                if name in self.feature_index_:
                    ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
                # ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], 1))
        return ' '.join(ffm)

    def transform(self, df):
        # val=[]
        # for k,v in self.feature_index_.items():
        #     val.append(v)
        # val.sort()
        # print(val)
        # print(self.field_index_)
        # print(self.feature_index_)
        # print (pd.Series({idx: row for idx, row in df.iterrows()})) # not shuffle
        return pd.Series({idx: self.transform_row_(row) for idx, row in df.iterrows()})

In [3]:
tr = LibsvmFormat(one_hot_feature)
user_ffm,field_sizes=tr.fit_transform(data)
user_ffm.to_csv('./data/data_libsvm/ffm_1.csv',index=False)
print (len(user_ffm))

#ytrain = np.array(train_.pop('col_3'))
#ytest = np.array(test.pop('col_3'))
ytrain = np.array([0,1,0])
ytest = np.array([1,0,1])
len_train=len(train_)
print (len(train_),len(test_))
with open('./data/data_libsvm/ffm_1.csv') as fin:
    f_train_out=open('./data/data_libsvm/train_ffm_1.csv','w')
    f_test_out = open('./data/data_libsvm/test_ffm_1.csv', 'w')
    for (i,line) in enumerate(fin):
        if i<int(len_train):
            f_train_out.write(str(ytrain[i])+' '+line)
        else:
            f_test_out.write(str(ytest[i-int(len_train)])+' ' +line)
    f_train_out.close()
    f_test_out.close()

col_0
col_1
col_2
col_4
[6, 6, 6, 5]
6
3 3


/students/julyedu_475082/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


#### model based

In [2]:
#导库
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy.sparse import coo_matrix
from sklearn.metrics import roc_auc_score

In [3]:
dtype = tf.float32
FIELD_SIZES = [29154, 777745, 6814, 666629]
#print (FIELD_SIZES)
#print (sum(FIELD_SIZES))
FIELD_OFFSETS = [sum(FIELD_SIZES[:i]) for i in range(len(FIELD_SIZES))]
INPUT_DIM = sum(FIELD_SIZES)
OUTPUT_DIM = 1
STDDEV = 1e-3 #0.001
MINVAL = -1e-3
MAXVAL = 1e-3

In [4]:
def libsvm_2_coo(libsvm_data, shape):
    coo_rows = []
    coo_cols = []
    coo_data = []
    n = 0
    for x, d in libsvm_data:
        coo_rows.extend([n] * len(x))
        coo_cols.extend(x)
        coo_data.extend(d)
        n += 1
    coo_rows = np.array(coo_rows)
    coo_cols = np.array(coo_cols)
    coo_data = np.array(coo_data)
    return coo_matrix((coo_data, (coo_rows, coo_cols)), shape=shape)
def read_data(file_name):
    X = []
    D = []
    y = []
    fin = pd.read_csv(file_name, header=None)
    fin = fin.values
    #print (fin.shape)
    for line in fin:
        X_i = []
        D_i = []
        line = line[0].strip().split()
        #print (line)
        if str(line[0]) == '0':
            y_i = float(0)
        else:
            y_i = float(1)
        for x in line[1:]:
            X_i.append(float(x.split(':')[1]))
            D_i.append(float(x.split(':')[2]))
        y.append(y_i)
        X.append(X_i)
        D.append(D_i)
    y = np.reshape(np.array(y), [-1])
    X = libsvm_2_coo(zip(X, D), (len(X), INPUT_DIM)).tocsr()
    return X, y
def shuffle(data):   #not used
    X, y = data
    ind = np.arange(X.shape[0])
    for i in range(7):
        np.random.shuffle(ind)
    return X[ind], y[ind]

In [5]:
def csr_2_input(csr_mat):
    if not isinstance(csr_mat, list):
        coo_mat = csr_mat.tocoo()
        indices = np.vstack((coo_mat.row, coo_mat.col)).transpose()
        values = csr_mat.data
        shape = csr_mat.shape
        return indices, values, shape
    else:
        inputs = []
        for csr_i in csr_mat:
            inputs.append(csr_2_input(csr_i))
        return inputs
def slice(csr_data, start=0, size=-1):
    if not isinstance(csr_data[0], list):
        if size == -1 or start + size >= csr_data[0].shape[0]:
            slc_data = csr_data[0][start:]
            slc_labels = csr_data[1][start:]
        else:
            slc_data = csr_data[0][start:start + size]
            slc_labels = csr_data[1][start:start + size]
    else:
        if size == -1 or start + size >= csr_data[0][0].shape[0]:
            slc_data = []
            for d_i in csr_data[0]:
                slc_data.append(d_i[start:])
            slc_labels = csr_data[1][start:]
        else:
            slc_data = []
            for d_i in csr_data[0]:
                slc_data.append(d_i[start:start + size])
            slc_labels = csr_data[1][start:start + size]
    return csr_2_input(slc_data), slc_labels
def split_data(data, skip_empty=True):
    fields = []
    for i in range(len(FIELD_OFFSETS) - 1):
        start_ind = FIELD_OFFSETS[i]
        end_ind = FIELD_OFFSETS[i + 1]
        if skip_empty and start_ind == end_ind:
            continue
        field_i = data[0][:, start_ind:end_ind]
        fields.append(field_i)
    fields.append(data[0][:, FIELD_OFFSETS[-1]:])
    return fields, data[1]

In [6]:
def init_var_map(init_vars, init_path=None):
    if init_path is not None:
        load_var_map = pkl.load(open(init_path, 'rb'))
        print('load variable map from', init_path, load_var_map.keys())
    var_map = {}
    for var_name, var_shape, init_method, dtype in init_vars:
        if init_method == 'zero':
            var_map[var_name] = tf.Variable(tf.zeros(var_shape, dtype=dtype), name=var_name, dtype=dtype)
        elif init_method == 'one':
            var_map[var_name] = tf.Variable(tf.ones(var_shape, dtype=dtype), name=var_name, dtype=dtype)
        elif init_method == 'normal':
            var_map[var_name] = tf.Variable(tf.random_normal(var_shape, mean=0.0, stddev=STDDEV, dtype=dtype),
                                            name=var_name, dtype=dtype)
        elif init_method == 'tnormal':
            var_map[var_name] = tf.Variable(tf.truncated_normal(var_shape, mean=0.0, stddev=STDDEV, dtype=dtype),
                                            name=var_name, dtype=dtype)
        elif init_method == 'uniform':
            var_map[var_name] = tf.Variable(tf.random_uniform(var_shape, minval=MINVAL, maxval=MAXVAL, dtype=dtype),
                                            name=var_name, dtype=dtype)
        elif init_method == 'xavier':
            maxval = np.sqrt(6. / np.sum(var_shape))
            minval = -maxval
            var_map[var_name] = tf.Variable(tf.random_uniform(var_shape, minval=minval, maxval=maxval, dtype=dtype),
                                            name=var_name, dtype=dtype)
        elif isinstance(init_method, int) or isinstance(init_method, float):
            var_map[var_name] = tf.Variable(tf.ones(var_shape, dtype=dtype) * init_method, name=var_name, dtype=dtype)
        elif init_method in load_var_map:
            if load_var_map[init_method].shape == tuple(var_shape):
                var_map[var_name] = tf.Variable(load_var_map[init_method], name=var_name, dtype=dtype)
            else:
                print('BadParam: init method', init_method, 'shape', var_shape, load_var_map[init_method].shape)
        else:
            print('BadParam: init method', init_method)
    return var_map
def activate(weights, activation_function):
    if activation_function == 'sigmoid':
        return tf.nn.sigmoid(weights)
    elif activation_function == 'softmax':
        return tf.nn.softmax(weights)
    elif activation_function == 'relu':
        return tf.nn.relu(weights)
    elif activation_function == 'tanh':
        return tf.nn.tanh(weights)
    elif activation_function == 'elu':
        return tf.nn.elu(weights)
    elif activation_function == 'none':
        return weights
    else:
        return weights
def get_optimizer(opt_algo, learning_rate, loss):
    if opt_algo == 'adaldeta':
        return tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'adagrad':
        return tf.train.AdagradOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'adam':
        return tf.train.AdamOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'ftrl':
        return tf.train.FtrlOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'gd':
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'padagrad':
        return tf.train.ProximalAdagradOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'pgd':
        return tf.train.ProximalGradientDescentOptimizer(learning_rate).minimize(loss)
    elif opt_algo == 'rmsprop':
        return tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
    else:
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [7]:
class Model:
    def __init__(self):
        self.sess = None
        self.X = None
        self.y = None
        self.layer_keeps = None
        self.vars = None
        self.keep_prob_train = None
        self.keep_prob_test = None

    def run(self, fetches, X=None, y=None, mode='train'):
            feed_dict = {}
            if type(self.X) is list:
                for i in range(len(X)):
                    feed_dict[self.X[i]] = X[i]
            else:
                feed_dict[self.X] = X
            if y is not None:
                feed_dict[self.y] = y
            if self.layer_keeps is not None:
                if mode == 'train':
                    feed_dict[self.layer_keeps] = self.keep_prob_train
                elif mode == 'test':
                    feed_dict[self.layer_keeps] = self.keep_prob_test
            return self.sess.run(fetches, feed_dict)

    def dump(self, model_path):
        var_map = {}
        for name, var in self.vars.iteritems():
            var_map[name] = self.run(var)
        pkl.dump(var_map, open(model_path, 'wb'))
        print('model dumped at', model_path)

#### models

In [8]:
class LR(Model):
    def __init__(self, input_dim=None, output_dim=1, init_path=None, opt_algo='gd', learning_rate=1e-2, l2_weight=0,
                 random_seed=None):
        Model.__init__(self)
        init_vars = [('w', [input_dim, output_dim], 'xavier', dtype),
                     ('b', [output_dim], 'zero', dtype)]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = tf.sparse_placeholder(dtype)
            self.y = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)

            w = self.vars['w']
            b = self.vars['b']
            xw = tf.sparse_tensor_dense_matmul(self.X, w)
            logits = tf.reshape(xw + b, [-1])
            self.y_prob = tf.sigmoid(logits)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(labels=self.y, logits=logits)) + \
                        l2_weight * tf.nn.l2_loss(xw)
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

class FM(Model):
    def __init__(self, input_dim=None, output_dim=1, factor_order=10, init_path=None, opt_algo='gd', learning_rate=1e-2,
                 l2_w=0, l2_v=0, random_seed=None):
        Model.__init__(self)
        init_vars = [('w', [input_dim, output_dim], 'xavier', dtype),
                     ('v', [input_dim, factor_order], 'xavier', dtype),
                     ('b', [output_dim], 'zero', dtype)]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = tf.sparse_placeholder(dtype)
            self.y = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)

            w = self.vars['w']
            v = self.vars['v']
            b = self.vars['b']

            X_square = tf.SparseTensor(self.X.indices, tf.square(self.X.values), tf.to_int64(tf.shape(self.X)))
            xv = tf.square(tf.sparse_tensor_dense_matmul(self.X, v))
            p = 0.5 * tf.reshape(
                tf.reduce_sum(xv - tf.sparse_tensor_dense_matmul(X_square, tf.square(v)), 1),
                [-1, output_dim])
            xw = tf.sparse_tensor_dense_matmul(self.X, w)
            logits = tf.reshape(xw + b + p, [-1])
            self.y_prob = tf.sigmoid(logits)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=self.y)) + \
                        l2_w * tf.nn.l2_loss(xw) + \
                        l2_v * tf.nn.l2_loss(xv)
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

class FNN(Model):
    def __init__(self, field_sizes=None, embed_size=10, layer_sizes=None, layer_acts=None, drop_out=None,
                 embed_l2=None, layer_l2=None, init_path=None, opt_algo='gd', learning_rate=1e-2, random_seed=None):
        Model.__init__(self)
        init_vars = []
        num_inputs = len(field_sizes)
        for i in range(num_inputs):
            init_vars.append(('embed_%d' % i, [field_sizes[i], embed_size], 'xavier', dtype))
        node_in = num_inputs * embed_size
        for i in range(len(layer_sizes)):
            init_vars.append(('w%d' % i, [node_in, layer_sizes[i]], 'xavier', dtype))
            init_vars.append(('b%d' % i, [layer_sizes[i]], 'zero', dtype))
            node_in = layer_sizes[i]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = [tf.sparse_placeholder(dtype) for i in range(num_inputs)]
            self.y = tf.placeholder(dtype)
            self.keep_prob_train = 1 - np.array(drop_out)
            self.keep_prob_test = np.ones_like(drop_out)
            self.layer_keeps = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)
            w0 = [self.vars['embed_%d' % i] for i in range(num_inputs)]
            xw = tf.concat([tf.sparse_tensor_dense_matmul(self.X[i], w0[i]) for i in range(num_inputs)], 1)
            l = xw

            for i in range(len(layer_sizes)):
                wi = self.vars['w%d' % i]
                bi = self.vars['b%d' % i]
                print(l.shape, wi.shape, bi.shape)
                l = tf.nn.dropout(activate(
                        tf.matmul(l, wi) + bi,
                        layer_acts[i]),
                    self.layer_keeps[i])

            l = tf.squeeze(l)
            self.y_prob = tf.sigmoid(l)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=self.y))
            if layer_l2 is not None:
                self.loss += embed_l2 * tf.nn.l2_loss(xw)
                for i in range(len(layer_sizes)):
                    wi = self.vars['w%d' % i]
                    self.loss += layer_l2[i] * tf.nn.l2_loss(wi)
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

class PNN1(Model):
    def __init__(self, field_sizes=None, embed_size=10, layer_sizes=None, layer_acts=None, drop_out=None,
                 embed_l2=None, layer_l2=None, init_path=None, opt_algo='gd', learning_rate=1e-2, random_seed=None):
        Model.__init__(self)
        init_vars = []
        num_inputs = len(field_sizes)
        for i in range(num_inputs):
            init_vars.append(('embed_%d' % i, [field_sizes[i], embed_size], 'xavier', dtype))
        num_pairs = int(num_inputs * (num_inputs - 1) / 2)
        node_in = num_inputs * embed_size + num_pairs
        # node_in = num_inputs * (embed_size + num_inputs)
        for i in range(len(layer_sizes)):
            init_vars.append(('w%d' % i, [node_in, layer_sizes[i]], 'xavier', dtype))
            init_vars.append(('b%d' % i, [layer_sizes[i]], 'zero', dtype))
            node_in = layer_sizes[i]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = [tf.sparse_placeholder(dtype) for i in range(num_inputs)]
            self.y = tf.placeholder(dtype)
            self.keep_prob_train = 1 - np.array(drop_out)
            self.keep_prob_test = np.ones_like(drop_out)
            self.layer_keeps = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)
            w0 = [self.vars['embed_%d' % i] for i in range(num_inputs)]
            xw = tf.concat([tf.sparse_tensor_dense_matmul(self.X[i], w0[i]) for i in range(num_inputs)], 1)
            xw3d = tf.reshape(xw, [-1, num_inputs, embed_size])

            row = []
            col = []
            for i in range(num_inputs-1):
                for j in range(i+1, num_inputs):
                    row.append(i)
                    col.append(j)
            # batch * pair * k
            p = tf.transpose(
                # pair * batch * k
                tf.gather(
                    # num * batch * k
                    tf.transpose(
                        xw3d, [1, 0, 2]),
                    row),
                [1, 0, 2])
            # batch * pair * k
            q = tf.transpose(
                tf.gather(
                    tf.transpose(
                        xw3d, [1, 0, 2]),
                    col),
                [1, 0, 2])
            p = tf.reshape(p, [-1, num_pairs, embed_size])
            q = tf.reshape(q, [-1, num_pairs, embed_size])
            ip = tf.reshape(tf.reduce_sum(p * q, [-1]), [-1, num_pairs])

            # simple but redundant
            # batch * n * 1 * k, batch * 1 * n * k
            # ip = tf.reshape(
            #     tf.reduce_sum(
            #         tf.expand_dims(xw3d, 2) *
            #         tf.expand_dims(xw3d, 1),
            #         3),
            #     [-1, num_inputs**2])
            l = tf.concat([xw, ip], 1)

            for i in range(len(layer_sizes)):
                wi = self.vars['w%d' % i]
                bi = self.vars['b%d' % i]
                l = tf.nn.dropout(activate(
                        tf.matmul(l, wi) + bi,
                        layer_acts[i]),
                    self.layer_keeps[i])

            l = tf.squeeze(l)
            self.y_prob = tf.sigmoid(l)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=self.y))
            if layer_l2 is not None:
                self.loss += embed_l2 * tf.nn.l2_loss(xw)
                for i in range(len(layer_sizes)):
                    wi = self.vars['w%d' % i]
                    self.loss += layer_l2[i] * tf.nn.l2_loss(wi)
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

class PNN2(Model):
    def __init__(self, field_sizes=None, embed_size=10, layer_sizes=None, layer_acts=None, drop_out=None,
                 embed_l2=None, layer_l2=None, init_path=None, opt_algo='gd', learning_rate=1e-2, random_seed=None,
                 layer_norm=True):
        Model.__init__(self)
        init_vars = []
        num_inputs = len(field_sizes)
        for i in range(num_inputs):
            init_vars.append(('embed_%d' % i, [field_sizes[i], embed_size], 'xavier', dtype))
        num_pairs = int(num_inputs * (num_inputs - 1) / 2)
        node_in = num_inputs * embed_size + num_pairs
        init_vars.append(('kernel', [embed_size, num_pairs, embed_size], 'xavier', dtype))
        for i in range(len(layer_sizes)):
            init_vars.append(('w%d' % i, [node_in, layer_sizes[i]], 'xavier', dtype))
            init_vars.append(('b%d' % i, [layer_sizes[i]], 'zero',  dtype))
            node_in = layer_sizes[i]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = [tf.sparse_placeholder(dtype) for i in range(num_inputs)]
            self.y = tf.placeholder(dtype)
            self.keep_prob_train = 1 - np.array(drop_out)
            self.keep_prob_test = np.ones_like(drop_out)
            self.layer_keeps = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)
            w0 = [self.vars['embed_%d' % i] for i in range(num_inputs)]
            xw = tf.concat([tf.sparse_tensor_dense_matmul(self.X[i], w0[i]) for i in range(num_inputs)], 1)
            xw3d = tf.reshape(xw, [-1, num_inputs, embed_size])

            row = []
            col = []
            for i in range(num_inputs - 1):
                for j in range(i + 1, num_inputs):
                    row.append(i)
                    col.append(j)
            # batch * pair * k
            p = tf.transpose(
                # pair * batch * k
                tf.gather(
                    # num * batch * k
                    tf.transpose(
                        xw3d, [1, 0, 2]),
                    row),
                [1, 0, 2])
            # batch * pair * k
            q = tf.transpose(
                tf.gather(
                    tf.transpose(
                        xw3d, [1, 0, 2]),
                    col),
                [1, 0, 2])
            # b * p * k
            p = tf.reshape(p, [-1, num_pairs, embed_size])
            # b * p * k
            q = tf.reshape(q, [-1, num_pairs, embed_size])
            # k * p * k
            k = self.vars['kernel']

            # batch * 1 * pair * k
            p = tf.expand_dims(p, 1)
            # batch * pair
            kp = tf.reduce_sum(
                # batch * pair * k
                tf.multiply(
                    # batch * pair * k
                    tf.transpose(
                        # batch * k * pair
                        tf.reduce_sum(
                            # batch * k * pair * k
                            tf.multiply(
                                p, k),
                            -1),
                        [0, 2, 1]),
                    q),
                -1)

            #
            # if layer_norm:
            #     # x_mean, x_var = tf.nn.moments(xw, [1], keep_dims=True)
            #     # xw = (xw - x_mean) / tf.sqrt(x_var)
            #     # x_g = tf.Variable(tf.ones([num_inputs * embed_size]), name='x_g')
            #     # x_b = tf.Variable(tf.zeros([num_inputs * embed_size]), name='x_b')
            #     # x_g = tf.Print(x_g, [x_g[:10], x_b])
            #     # xw = xw * x_g + x_b
            #     p_mean, p_var = tf.nn.moments(op, [1], keep_dims=True)
            #     op = (op - p_mean) / tf.sqrt(p_var)
            #     p_g = tf.Variable(tf.ones([embed_size**2]), name='p_g')
            #     p_b = tf.Variable(tf.zeros([embed_size**2]), name='p_b')
            #     # p_g = tf.Print(p_g, [p_g[:10], p_b])
            #     op = op * p_g + p_b

            l = tf.concat([xw, kp], 1)
            for i in range(len(layer_sizes)):
                wi = self.vars['w%d' % i]
                bi = self.vars['b%d' % i]
                l = tf.nn.dropout(activate(
                        tf.matmul(l, wi) + bi,
                        layer_acts[i]),
                    self.layer_keeps[i])

            l = tf.squeeze(l)
            self.y_prob = tf.sigmoid(l)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=self.y))
            if layer_l2 is not None:
                self.loss += embed_l2 * tf.nn.l2_loss(xw)#tf.concat(w0, 0))
                for i in range(len(layer_sizes)):
                    wi = self.vars['w%d' % i]
                    self.loss += layer_l2[i] * tf.nn.l2_loss(wi)
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)
            
class CCPM(Model):
    def __init__(self, field_sizes=None, embed_size=10, filter_sizes=None, layer_acts=None, drop_out=None,
                 init_path=None, opt_algo='gd', learning_rate=1e-2, random_seed=None):
        Model.__init__(self)
        init_vars = []
        num_inputs = len(field_sizes)
        for i in range(num_inputs):
            init_vars.append(('embed_%d' % i, [field_sizes[i], embed_size], 'xavier', dtype))
        init_vars.append(('f1', [embed_size, filter_sizes[0], 1, 2], 'xavier', dtype))
        init_vars.append(('f2', [embed_size, filter_sizes[1], 2, 2], 'xavier', dtype))
        init_vars.append(('w1', [2 * 3 * embed_size, 1], 'xavier', dtype))
        init_vars.append(('b1', [1], 'zero', dtype))

        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = [tf.sparse_placeholder(dtype) for i in range(num_inputs)]
            self.y = tf.placeholder(dtype)
            self.keep_prob_train = 1 - np.array(drop_out)
            self.keep_prob_test = np.ones_like(drop_out)
            self.layer_keeps = tf.placeholder(dtype)
            self.vars = init_var_map(init_vars, init_path)
            w0 = [self.vars['embed_%d' % i] for i in range(num_inputs)]
            xw = tf.concat([tf.sparse_tensor_dense_matmul(self.X[i], w0[i]) for i in range(num_inputs)], 1)
            l = xw

            l = tf.transpose(tf.reshape(l, [-1, num_inputs, embed_size, 1]), [0, 2, 1, 3])
            f1 = self.vars['f1']
            l = tf.nn.conv2d(l, f1, [1, 1, 1, 1], 'SAME')
            l = tf.transpose(max_pool_4d(
                    tf.transpose(l, [0, 1, 3, 2]),
                    int(num_inputs / 2)),
                [0, 1, 3, 2])
            f2 = self.vars['f2']
            l = tf.nn.conv2d(l, f2, [1, 1, 1, 1], 'SAME')
            l = tf.transpose(max_pool_4d(
                    tf.transpose(l, [0, 1, 3, 2]), 3),
                [0, 1, 3, 2])
            l = tf.nn.dropout(activate(
                    tf.reshape(l, [-1, embed_size * 3 * 2]),
                    layer_acts[0]),
                self.layer_keeps[0])
            w1 = self.vars['w1']
            b1 = self.vars['b1']
            l = tf.matmul(l, w1) + b1

            l = tf.squeeze(l)
            self.y_prob = tf.sigmoid(l)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=l, labels=self.y))
            self.optimizer = get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

#### training

In [9]:
train_file = './data/data_libsvm/train_ffm.csv'
test_file = './data/data_libsvm/test_ffm.csv'

input_dim = INPUT_DIM

# 读取数据
train_data = read_data(train_file)
#train_data = shuffle(train_data)
test_data = read_data(test_file)

# 输出数据信息维度
if train_data[1].ndim > 1:
    print('label must be 1-dim')
    exit(0)
print('read finish')
print('train data size:', train_data[0].shape)
print('test data size:', test_data[0].shape)

# 训练集与测试集
train_size = train_data[0].shape[0]
test_size = test_data[0].shape[0]
num_feas = len(FIELD_SIZES)

# 超参数设定
min_round = 1
num_round = 200
early_stop_round = 5
# train + val
batch_size = 1024

field_sizes = FIELD_SIZES
field_offsets = FIELD_OFFSETS
print(field_sizes)
print(field_offsets)

algo = 'pnn2'

if algo in {'fnn','anfm','amlp','ccpm','pnn1','pnn2'}:
    train_data = split_data(train_data)
    test_data = split_data(test_data)
    tmp = []
    for x in field_sizes:
        if x > 0:
            tmp.append(x)
    field_sizes = tmp
    print('remove empty fields', field_sizes)

if algo == 'lr':
    lr_params = {
        'input_dim': input_dim,
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'l2_weight': 0,
        'random_seed': 0
    }
    print(lr_params)
    model = LR(**lr_params)
elif algo == 'fm':
    fm_params = {
        'input_dim': input_dim,
        'factor_order': 10,
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'l2_w': 0,
        'l2_v': 0,
    }
    print(fm_params)
    model = FM(**fm_params)
elif algo == 'fnn':
    fnn_params = {
        'field_sizes': field_sizes,
        'embed_size': 10,
        'layer_sizes': [500, 1],
        'layer_acts': ['relu', None],
        'drop_out': [0, 0],
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'embed_l2': 0,
        'layer_l2': [0, 0],
        'random_seed': 0
    }
    print(fnn_params)
    model = FNN(**fnn_params)
elif algo == 'pnn1':
    pnn1_params = {
        'field_sizes': field_sizes,
        'embed_size': 10,
        'layer_sizes': [500, 1],
        'layer_acts': ['relu', None],
        'drop_out': [0, 0],
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'embed_l2': 0,
        'layer_l2': [0, 0],
        'random_seed': 0
    }
    print(pnn1_params)
    model = PNN1(**pnn1_params)
elif algo == 'pnn2':
    pnn2_params = {
        'field_sizes': field_sizes,
        'embed_size': 10,
        'layer_sizes': [500, 1],
        'layer_acts': ['relu', None],
        'drop_out': [0, 0],
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'embed_l2': 0,
        'layer_l2': [0., 0.],
        'random_seed': 0,
        'layer_norm': True,
    }
    print(pnn2_params)
    model = PNN2(**pnn2_params)
elif algo == 'ccpm':
    ccpm_params = {
        'field_sizes': field_sizes,
        'embed_size': 10,
        'filter_sizes': [5, 3],
        'layer_acts': ['relu'],
        'drop_out': [0],
        'opt_algo': 'gd',
        'learning_rate': 0.1,
        'random_seed': 0
    }
    print(ccpm_params)
    model = CCPM(**ccpm_params)

def train(model):
    history_score = []
    for i in range(num_round):
        fetches = [model.optimizer, model.loss]
        #if model.model_file is not None:
        #	model.saver.restore(model.sess,model.model_file)
        if batch_size > 0:
            ls = []
            print('[%d]\ttraining...' % i)
            for j in range(int(train_size / batch_size + 1)):
                X_i, y_i = slice(train_data, j * batch_size, batch_size)
                _, l = model.run(fetches, X_i, y_i)
                ls.append(l)
        elif batch_size == -1:
            X_i, y_i = slice(train_data)
            print (X_i.shape)
            print (y_i.shape)
            _, l = model.run(fetches, X_i, y_i)
            ls = [l]
        #model.saver.save(model.sess,'./Save/train.ckpt')
        train_preds = []
        print('[%d]\tevaluating...' % i)
        for j in range(int(train_size / 10000 + 1)):
            X_i, _ = slice(train_data, j * 10000, 10000)
            preds = model.run(model.y_prob, X_i, mode='test')
            train_preds.extend(preds)
        test_preds = []
        for j in range(int(test_size / 10000 + 1)):
            X_i, _ = slice(test_data, j * 10000, 10000)
            preds = model.run(model.y_prob, X_i, mode='test')
            test_preds.extend(preds)
        #print('train size', len(train_data[1]), len(train_preds))
        #print('test size', len(test_data[1]), len(test_preds))
        train_score = roc_auc_score(train_data[1], train_preds)
        test_score = roc_auc_score(test_data[1], test_preds)
        print('[%d]\tloss (with l2 norm):%f\ttrain-auc: %f\ttest-auc: %f' % (i, np.mean(ls), train_score, test_score))
        history_score.append(test_score)
        if i > min_round and i > early_stop_round:
            if np.argmax(history_score) == i - early_stop_round and history_score[-1] - history_score[
                        -1 * early_stop_round] < 1e-5:
                print('early stop\nbest iteration:\n[%d]\ttest-auc: %f' % (
                    np.argmax(history_score), np.max(history_score)))
                break

train(model)

read finish
train data size: (2196681, 1480342)
test data size: (549171, 1480342)
[29154, 777745, 6814, 666629]
[0, 29154, 806899, 813713]
remove empty fields [29154, 777745, 6814, 666629]
{'embed_size': 10, 'embed_l2': 0, 'layer_norm': True, 'learning_rate': 0.1, 'layer_acts': ['relu', None], 'drop_out': [0, 0], 'layer_l2': [0.0, 0.0], 'field_sizes': [29154, 777745, 6814, 666629], 'random_seed': 0, 'layer_sizes': [500, 1], 'opt_algo': 'gd'}


/usr/local/python3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[0]	training...
[0]	evaluating...
[0]	loss (with l2 norm):0.112005	train-auc: 0.607107	test-auc: 0.604898
[1]	training...
[1]	evaluating...
[1]	loss (with l2 norm):0.106886	train-auc: 0.639231	test-auc: 0.635022
[2]	training...
[2]	evaluating...
[2]	loss (with l2 norm):0.106025	train-auc: 0.664341	test-auc: 0.658876
[3]	training...
[3]	evaluating...
[3]	loss (with l2 norm):0.104631	train-auc: 0.680163	test-auc: 0.674214
[4]	training...
[4]	evaluating...
[4]	loss (with l2 norm):0.103396	train-auc: 0.692265	test-auc: 0.685785
[5]	training...
[5]	evaluating...
[5]	loss (with l2 norm):0.102599	train-auc: 0.701813	test-auc: 0.695027
[6]	training...
[6]	evaluating...
[6]	loss (with l2 norm):0.102054	train-auc: 0.707658	test-auc: 0.700470
[7]	training...
[7]	evaluating...
[7]	loss (with l2 norm):0.101638	train-auc: 0.711692	test-auc: 0.704182
[8]	training...
[8]	evaluating...
[8]	loss (with l2 norm):0.101297	train-auc: 0.715031	test-auc: 0.707154
[9]	training...
[9]	evaluating...
[9]	loss (wi

##### OPNN:loss (with l2 norm):0.078839	train-auc: 0.901104	test-auc: 0.755782

read finish
train data size: (2196681, 1480342)
test data size: (549171, 1480342)
[29154, 777745, 6814, 666629]
[0, 29154, 806899, 813713]
remove empty fields [29154, 777745, 6814, 666629]
{'layer_acts': ['relu', None], 'layer_l2': [0, 0], 'field_sizes': [29154, 777745, 6814, 666629], 'opt_algo': 'gd', 'embed_l2': 0, 'drop_out': [0, 0], 'learning_rate': 0.1, 'layer_sizes': [500, 1], 'embed_size': 10, 'random_seed': 0}


/usr/local/python3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[0]	training...
[0]	evaluating...
[0]	loss (with l2 norm):0.111966	train-auc: 0.592476	test-auc: 0.592175
[1]	training...
[1]	evaluating...
[1]	loss (with l2 norm):0.107018	train-auc: 0.636945	test-auc: 0.634140
[2]	training...
[2]	evaluating...
[2]	loss (with l2 norm):0.106392	train-auc: 0.661471	test-auc: 0.656811
[3]	training...
[3]	evaluating...
[3]	loss (with l2 norm):0.105149	train-auc: 0.677856	test-auc: 0.672569
[4]	training...
[4]	evaluating...
[4]	loss (with l2 norm):0.103789	train-auc: 0.690366	test-auc: 0.684451
[5]	training...
[5]	evaluating...
[5]	loss (with l2 norm):0.102855	train-auc: 0.699777	test-auc: 0.693311
[6]	training...
[6]	evaluating...
[6]	loss (with l2 norm):0.102236	train-auc: 0.706545	test-auc: 0.699690
[7]	training...
[7]	evaluating...
[7]	loss (with l2 norm):0.101774	train-auc: 0.711055	test-auc: 0.703895
[8]	training...
[8]	evaluating...
[8]	loss (with l2 norm):0.101400	train-auc: 0.714602	test-auc: 0.707043
[9]	training...
[9]	evaluating...
[9]	loss (wi

##### IPNN:loss (with l2 norm):0.079100	train-auc: 0.901606	test-auc: 0.759004

read finish
train data size: (2196681, 1480342)
test data size: (549171, 1480342)
[29154, 777745, 6814, 666629]
[0, 29154, 806899, 813713]
remove empty fields [29154, 777745, 6814, 666629]
{'layer_acts': ['relu', None], 'layer_l2': [0, 0], 'field_sizes': [29154, 777745, 6814, 666629], 'opt_algo': 'gd', 'embed_l2': 0, 'drop_out': [0, 0], 'learning_rate': 0.1, 'layer_sizes': [500, 1], 'embed_size': 10, 'random_seed': 0}
(?, 40) (40, 500) (500,)
(?, 500) (500, 1) (1,)
[0]	training...
[0]	evaluating...
[0]	loss (with l2 norm):0.111966	train-auc: 0.592775	test-auc: 0.592477
[1]	training...
[1]	evaluating...
[1]	loss (with l2 norm):0.107015	train-auc: 0.637218	test-auc: 0.634408
[2]	training...
[2]	evaluating...
[2]	loss (with l2 norm):0.106383	train-auc: 0.661668	test-auc: 0.657007
[3]	training...
[3]	evaluating...
[3]	loss (with l2 norm):0.105134	train-auc: 0.678028	test-auc: 0.672741
[4]	training...
[4]	evaluating...
[4]	loss (with l2 norm):0.103775	train-auc: 0.690502	test-auc: 0.684586


##### FNN:loss (with l2 norm):0.078925	train-auc: 0.902585	test-auc: 0.758936

read finish
train data size: (2196681, 1480342)
test data size: (549171, 1480342)
[29154, 777745, 6814, 666629]
[0, 29154, 806899, 813713]
{'l2_w': 0, 'factor_order': 10, 'l2_v': 0, 'learning_rate': 0.1, 'input_dim': 1480342, 'opt_algo': 'gd'}
[0]	training...
[0]	evaluating...
[0]	loss (with l2 norm):0.118525	train-auc: 0.703039	test-auc: 0.688827
[1]	training...
[1]	evaluating...
[1]	loss (with l2 norm):0.106234	train-auc: 0.719189	test-auc: 0.700400
[2]	training...
[2]	evaluating...
[2]	loss (with l2 norm):0.105905	train-auc: 0.723584	test-auc: 0.703765
[3]	training...
[3]	evaluating...
[3]	loss (with l2 norm):0.105621	train-auc: 0.725523	test-auc: 0.705091
[4]	training...
[4]	evaluating...
[4]	loss (with l2 norm):0.105372	train-auc: 0.726708	test-auc: 0.705779
[5]	training...
[5]	evaluating...
[5]	loss (with l2 norm):0.105151	train-auc: 0.727435	test-auc: 0.706158
[6]	training...
[6]	evaluating...
[6]	loss (with l2 norm):0.104953	train-auc: 0.728015	test-auc: 0.706617
[7]	training..

##### FM:loss (with l2 norm):0.098486	train-auc: 0.765485	test-auc: 0.727582

##### LR: [199]	loss (with l2 norm):0.098486	train-auc: 0.765473	test-auc: 0.727587